In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import ListedColormap
import re
import emoji
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk import pos_tag, pos_tag_sents
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
import csv

In [7]:
from flair.models import TextClassifier
from flair.data import Sentence

In [10]:
df_user_reviews = pd.read_csv("yelp_reviews_users_Phila_final.csv")

In [11]:
df_user_reviews_copy = df_user_reviews

In [12]:
# Step 1: Load the pre-trained sentiment analysis model
classifier = TextClassifier.load('en-sentiment')

# Step 2: Function to get sentiment score on a scale of -1 to 1
def get_sentiment_score(review):
    # Step 3: Create a sentence object
    sentence = Sentence(review)
    
    # Step 4: Predict sentiment
    classifier.predict(sentence)
    
    # Step 5: Get the sentiment score
    sentiment_score = sentence.labels[0].score  # The score is between 0 and 1 for positive sentiment
    
    # Convert to a continuous scale from -1 to 1 (positive sentiment is a score between 0 and 1)
    if sentence.labels[0].value == 'POSITIVE':
        return 2 * sentiment_score - 1  # Mapping the [0, 1] range to [-1, 1]
    else:
        return 1 - 2 * sentiment_score  # For negative sentiment, map to [-1, 0]


2024-11-12 21:13:17,300 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to C:\Users\LO3B19~1\Temp\tmp4b8uehjn


100%|███████████████████████████████████████████████████████████████████████████████| 253M/253M [00:12<00:00, 21.5MB/s]

2024-11-12 21:13:30,008 copying C:\Users\LO3B19~1\Temp\tmp4b8uehjn to cache at C:\Users\amrizvi2\.flair\models\sentiment-en-mix-distillbert_4.pt


2024-11-12 21:13:31,582 removing temp file C:\Users\LO3B19~1\Temp\tmp4b8uehjn


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
# Example usage
review = "The food was great, but the service was awful."
sentiment_score = get_sentiment_score(review)
print(f"Sentiment score: {sentiment_score}")

Sentiment score: -0.9622268676757812


In [14]:
def cleanText(text):
    # Making words lowercase
    text = text.lower()

    # Replacing instances of multiple whitespaces with single whitespaces and removing leading and trailing whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Removing emojis
    text = emoji.demojize(text)

    # Removing URLS
    text = re.sub(r'\s*http(s?)\S+|\s*www\S+', '', text)

    # Removing Emails
    text = re.sub(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9]+\.[a-zA-Z]{2,}', '', text)

    # Removing special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    return text

In [19]:
df_user_reviews['review_cleaned'] = df_user_reviews['review'].apply(cleanText)

0        there is absolutely nothing wrong with this re...
1        great place for brunch portions on certain dis...
2        my fiance loves french food so for her birthda...
3        delicious food great ambiance excellent servic...
4        i met a friend here for lunch and was disappoi...
                               ...                        
90386    the food was mediocre coming from a middle eas...
90387    if youre looking turkish spot in the city papr...
90388    great authentic turkish food i trued paprica f...
90389    delicious turkish dining the chicken shish keb...
90390    paprica is very cosmopolitan place where you w...
Name: review_cleaned, Length: 90391, dtype: object

In [ ]:
df_user_reviews['review_cleaned']

In [24]:
df_user_reviews.columns

Index(['business_id', 'restaurant_name', 'review_id', 'user_id', 'user_rating',
       'review', 'review_date', 'user_name', 'user_review_count',
       'user_average_rating', 'useful_user_review_count', 'review_cleaned'],
      dtype='object')

In [27]:
df_user_reviews['polarity'] = df_user_reviews.iloc[12:10000].applymap(get_sentiment_score)

C:\Users\local_amrizvi2\Temp\ipykernel_37600\1717291655.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_user_reviews['polarity'] = df_user_reviews.iloc[12:10000].applymap(get_sentiment_score)


KeyboardInterrupt: 

In [ ]:
df_user_reviews['polarity']

In [ ]:
# Plotting the distribution of sentiment scores computed using vaderSentiment
# Histogram
fig = px.histogram(df_user_reviews['polarity'], nbins=20, color_discrete_sequence=['darkred'], title = 'User Reviews Sentiment Score Distribution<br>Flair')

# Remove legend
fig.update_traces(showlegend=False)

# Customize the layout if needed
fig.update_layout(
    xaxis_title = 'Sentiment Scores',
    yaxis_title = 'Number of User Reviews',
    bargap = 0,
    xaxis_showgrid=True
)

# Show the plot
fig.show()

In [ ]:
# Computing average sentiment scores from vaderSentiment and TextBlob for each business_id (restaurant)
avg_sentiment = df_user_reviews.groupby(by = ['business_id'])[['polarity']].mean().reset_index()
avg_sentiment.columns = ['business_id', 'avg_sentiment_flair']

In [ ]:
## Labeling whether each tweet is positive, negative, or neutral for flairrSentiment
# Labels
labels = ['Positive', 'Neutral', 'Negative']
# Conditions for flairSentiment
conditions_flair = [
    (df_user_reviews['polarity'] > 0.1), #Positive 
    (df_user_reviews['polarity'] >= -0.1) & (df_user_reviews['polarity'] <= 0.1), #Neutral
    (df_user_reviews['polarity'] < -0.1) #Negative
]

df_user_reviews['sentiment_label_flair'] = np.select(conditions_flair, labels)

In [ ]:
df_user_reviews['sentiment_label_flair'].value_counts(dropna = False)

In [ ]:
# Counting the number of words in each review
df_user_reviews['review_word_count'] = df_user_reviews['review'].apply(lambda x: len([word for word in word_tokenize(x) if word.isalnum()]))

In [ ]:
## Normalizing variables/factors that contribute to the relative importance of a sentiment score
# Normalized user_review_count for each business_id (restaurant); range is [0, 1]
df_user_reviews['norm_user_review_count'] = df_user_reviews.groupby(by = ['business_id'])['user_review_count'].transform(lambda x: x / x.max())

# Normalized useful_user_review_count (number of votes that other users have cast to indicate if the review was helpful) 
# for each business_id (restaurant); range is [0, 1]
df_user_reviews['norm_useful_user_review_count'] = df_user_reviews.groupby(by = ['business_id'])['useful_user_review_count'].transform(lambda x: x / x.max())

# Normalized word count for each business_id (restaurant); range is [0, 1]
df_user_reviews['norm_review_word_count'] = df_user_reviews.groupby(by = ['business_id'])['review_word_count'].transform(lambda x: x / x.max())

# Normalized user rating; range is [0, 1]; closer to 0 means much lower than average rating and 1 means much higher than average rating
min_max_scaler = MinMaxScaler() #scaling deviations to [0,1] range using MinMaxScaler 
df_user_reviews['norm_user_rating'] = min_max_scaler.fit_transform((df_user_reviews['user_rating'] - df_user_reviews['user_average_rating']).values.reshape(-1, 1))


In [ ]:
#### Calculating weighted average sentiment score for each business_id (restaurant)
## Assign weights to each variable/factor (you can adjust these based on importance)
weight_user_review_count = 0.2
weight_useful_user_review_count = 0.35
weight_review_word_count = 0.1
weight_user_rating = 0.35

## Calculated weighted sum of variables/factors
df_user_reviews['sentiment_score_weight'] = weight_user_review_count * df_user_reviews['norm_user_review_count'] + \
                                            weight_useful_user_review_count * df_user_reviews['norm_useful_user_review_count'] + \
                                            weight_review_word_count * df_user_reviews['norm_review_word_count'] + \
                                            weight_user_rating * df_user_reviews['norm_user_rating']

## Calculating weighted average sentiment scores for each business_id (restaurant)
# flairSentiment weighted average sentiment score 
weighted_avg_sentiment_flair = df_user_reviews.groupby('business_id').apply(
    lambda x: (x['sentiment_score_weight'] * x['polarity']).sum() / x['sentiment_score_weight'].sum()
).reset_index(name = 'weighted_avg_sentiment_flair')


In [ ]:
## Adding the number of positive, neutral, and negative tweets to weighted_avg_sentiment_flair
sentiment_label_count_flair = pd.crosstab(df_user_reviews['business_id'], df_user_reviews['sentiment_label_flair']).reset_index() 
sentiment_label_count_flair.columns = ['business_id', 'negative_review_count_flair', 'neutral_review_count_flair', 'positive_review_count_flair']

In [ ]:
## Merging everything together
restaurants_sentiment_df = pd.merge(avg_sentiment, weighted_avg_sentiment_flair, how = 'inner', on = 'business_id')
restaurants_sentiment_df = pd.merge(restaurants_sentiment_df, sentiment_label_count_flair, how = 'inner', on = 'business_id')

In [ ]:
restaurants_sentiment_df